In [1]:
#!/usr/bin/env python
#coding=utf-8

In [4]:
# urllib 為收集了多個用到URL的套件:
    # urllib.request打開和讀取URL
    # urllib.error包含urllib.request拋出的異常
    # urllib.parse用於解析URL
    # urllib.robotparser用於解析robots.txt文件
from urllib.request import urlopen

# 一個HTML解析器，可以處理各種奇怪的HTML
from bs4 import BeautifulSoup

# os 套件提供了非常豐富的方法用來處理文件和目錄
import os

# json.dumps 用於將 Python 對象編碼成 JSON 字符串
import json

# 建立各種 HTTP 請求，從網頁伺服器上取得想要的資料
import requests

# 使 Python 語言擁有全部的正則表達式功能
import re

# 存檔成CSV用 
import pandas as pd

import csv

# 先建立所有連結的集合set()

目標: 先搜尋1 to 1900 (搜尋值)，將每頁"食物名"連結中各"食用量"的連結中的營養成分做成set() (集合,內文字為唯一值)

流程：

    第一部分：得所有第一層之連結（頁數連結）。
        1. 建立"頁數"連結 = 空set()。
        2. 做個搜尋值迴圈(while:)得各頁數連結 
            **使用if else做無效搜尋職或頁數之停損點。
        4. 將上述順序倒過來迴圈，並加入set()。
        5. 將set()轉換為list()，供第二部分使用。
        6. 將set()使用sorted依順序後存檔（csv）。
    
    第二部分：得第二層"食物名"連結。
        1. 建立"食物名"連結 = 空set()
        2. 使用for迴圈，從"頁數"set()得"食物名"連結。
        3. 將set()使用sorted依順序後存檔（csv），供第三部分使用。
                    
    第三部分：得第三層可能隱藏之連結。
        1. if:各"食物名"連結內有各"食用量"之連結，則得各"食用量"連結，else:僅得預設連結(可用len(網頁)為判別。並加入set()
            **因某些食物名連結為error: 做try except以跳過無效連結。
        2. 將set()使用sorted依順序後存檔（csv），以供第四部份使用。
    
    第四部分：爬蟲。
        1. 創字典。
        2. 將list()，以迴圈將品牌、食物名、營養成分、資料來源等，放入字典。        
        
    第五部分：存檔。
        1. 將不符合資料夾、檔案命名格式之文字轉換為""。
        2. 將食物名、營養成分、資料來源設成json格式儲存至指定資料夾。    

## 第一部分：得所有第一層之連結。

In [ ]:
# 網頁原本的樣子 https://www.fatsecret.cn/%E7%83%AD%E9%87%8F%E8%90%A5%E5%85%BB/search?q=0&pg=0
mobile = "https://mobile.fatsecret.cn/%E7%83%AD%E9%87%8F%E8%90%A5%E5%85%BB/search?q="
website = "https://www.fatsecret.cn/%E7%83%AD%E9%87%8F%E8%90%A5%E5%85%BB/search?q="
pg = "&pg="
pageLinks_set = set()
pageLinks_csv = [] # 將起始"全部頁數連結"為空字串

################## 設數字 #########################
q = 0 #設搜尋起始數字
count = 100
##################################################

print("開始抓第一部分：第一層之所有連結")

while True:
    pgNumber = 0 #設每一搜尋值之起始頁數
    
    # 超過搜尋頁數時，網頁會跳轉成 "mobile" 手機版網頁，所以先以此做為檢查依據
    web = mobile + str(q) + pg + str(pgNumber) 
    response = requests.get(web)  # 輸入cookie及內容
    html = BeautifulSoup(response.text) # 如果是requests, 多一個步驟, .text欄位拿出

    if count > 101: # 防止無限迴圈 # 當累積"超過頁數"3次時，表示搜尋值及頁數皆到盡頭，沒東西可抓了
        print("無搜尋值：" + str(q-100) + "到底了！")
        pageLinks_csv.append(("無搜尋值：" + str(q-100) + "到底了！" , " "))
        break   

    # 若該搜尋值僅有1頁，抓連結外，直接將搜尋值+1，回外層迴圈
    elif len(html.find_all("td", align="center")) == 0:
        print("搜尋值：" + str(q) + "，僅有1頁")

        websiteLink = website + str(q) + pg + str(pgNumber) 
        pageLinks_set.add(websiteLink)
        pageLinks_csv.append(("搜尋值：" + str(q) + "，僅有1頁：", websiteLink))
        print(websiteLink)
        print("")
        
        count = 0
        q += 1
                
    else:
        while True:
            # 抓該頁連結連結
            web = mobile + str(q) + pg + str(pgNumber) 
            response = requests.get(web)  
            html = BeautifulSoup(response.text) 

            # 如果超過該搜尋值之頁數則break，回外層迴圈
            if ((html.find_all("td", align="center")[0].text).split('\"')[0])  == '没有找到':
                print("搜尋值:" + str(q) + "並無第"+ str(pgNumber+1) + "頁，超過該搜尋值之頁數")
                print("")
                
                q += 1
                count += 1
                break                
            
            # 進此，表示該搜尋值有其他頁
            else:
                print("搜尋值："+ str(q) +"，第" + str(pgNumber) +"頁" )     
                websiteLink = website + str(q) + pg + str(pgNumber) 
                
                print(websiteLink)
                pageLinks_set.add(websiteLink)
                pageLinks_csv.append(("搜尋值："+ str(q) + "頁數：" + str(pgNumber) + "：", websiteLink))
                
                count = 0
                pgNumber += 1
                

pageLinks_set = sorted(pageLinks_set) # 依順序排序
pageLinks_csv = sorted(pageLinks_csv)

# 儲存檔案
# 如果沒有該檔案名，創資料夾
dn = "fatsecret_food"
if not os.path.exists(dn):   
    os.makedirs(dn)    

# 將pageLinks_scv存檔
print("頁數連結存檔中...")

my_df = pd.DataFrame(pageLinks_set)
my_df.to_csv(dn + "/" + 'WebLinks_fatsecret_1_pageLinks_set', index=False, header=False, encoding="utf-8")

my_df = pd.DataFrame(pageLinks_csv)
my_df.to_csv(dn + "/" + 'WebLinks_fatsecret_1_pageLinks_scv', index=False, header=False, encoding="utf-8")

print("頁數連結存檔（csv）完成")
print("="*80)

    

# 第二部分：得第二層"食物名"連結。

In [ ]:
foodLinks_set = set() # 建立"食物名"連結集合
foodLinks_csv = set()

for link in pageLinks_set:
    
    pageLinks = link
    response = requests.get(pageLinks)
    html = BeautifulSoup(response.text) 
    html_foodLinks = html.find_all("a", class_ = "prominent")

    for i in range (len(html_foodLinks)):
        foodName = html_foodLinks[i].text
        foodLinks = "https://www.fatsecret.cn/" + html_foodLinks[i]["href"]
        foodLinks_set.add((foodLinks))
        foodLinks_csv.add((foodName, foodLinks))
        print(foodName)
        print(foodLinks)
        print("")


foodLinks_set = sorted(foodLinks_set) # 依順序排序
foodLinks_csv = list(sorted(foodLinks_csv))

# 儲存檔案
# 如果沒有該檔案名，創資料夾
dn = "fatsecret_food"
if not os.path.exists(dn):   
    os.makedirs(dn)    


# 將foodLinks_scv存檔
print("食物名連結存檔中...")

my_df = pd.DataFrame(foodLinks_set)
my_df.to_csv(dn + "/" + 'WebLinks_fatsecret_2_foodLinks_set', index=False, header=False, encoding="utf-8")

my_df = pd.DataFrame(foodLinks_csv)
my_df.to_csv(dn + "/" + 'WebLinks_fatsecret_2_foodLinks_csv', index=False, header=False, encoding="utf-8")

print("食物名連結存檔（csv）完成")
print("="*80)

# 第三部分：得第三層可能隱藏之連結

#### 範例連結：
僅有預設連結（內無其他"食用量"之連結）：　
    1. 有品牌：
https://www.fatsecret.cn/%E7%83%AD%E9%87%8F%E8%90%A5%E5%85%BB/%E7%A7%91%E8%BF%AA/%E7%BA%AF%E7%89%9B%E5%A5%B6100/100%E6%AF%AB%E5%8D%87
連結內有其他"食用量"連結 ：
    1. 無品牌：
https://www.fatsecret.cn/%E7%83%AD%E9%87%8F%E8%90%A5%E5%85%BB/%E6%99%AE%E9%80%9A%E7%9A%84/%E6%B3%95%E5%9B%BD%E5%8D%A1%E9%97%A8%E5%9F%B9%E5%B0%94(camembert)%E5%A5%B6%E9%85%AA

In [ ]:
intakeLinks_set = set() # 設起始"食用量"連結為空字串
errorLinks_set = set() # 設"食物名"連結內為錯誤，為空字串

for iLink in foodLinks_set:
    try: 
        foodLinks = iLink
        response = requests.get(foodLinks)
        html = BeautifulSoup(response.text)  
        html_intakeLinks = html.find_all("a", href = re.compile("portionamount=")) #解析網頁
        # intakeLinks = "https://www.fatsecret.cn/" + html_intakeLinks[0]["href"] # 取得"食物名"連結
        

        if len(html_intakeLinks)  == 0: # foodLink 內僅有預設"食用量"連結

            html = BeautifulSoup(response.text) 
            brand = html.find_all("div", class_ = "breadcrumb_link") # 該食物有品牌，為[2]；若無，為""
            size =  html.find_all("div", class_ = "serving_size_value") # 食用量[0]
            intake =  html.find_all("div", class_ = "breadcrumb_noLink") # 食物名
            
            # 以len(brand判斷該食物是否有"品牌"
            if len(brand) == 3: # 有品牌
                intakeLinks_set.add((brand[2].text, intake[0].text, size[0].text, foodLinks))
                print("品牌："+ brand[2].text + "，食物名：" + intake[0].text + 
                      "，食用量：" + size[0].text + "，網址：", foodLinks)

            else: # 無"品牌
                intakeLinks_set.add(("NULL", intake[0].text, size[0].text, foodLinks))
                print("品牌："+ "NULL" + "，食物名：" + intake[0].text + 
                      "，食用量：" + size[0].text + "，網址：",foodLinks)
                    
        if len(html_intakeLinks) != 0:

            for i in range (0, len(html_intakeLinks), 2): #"食用量"與"卡路里"為同樣連結，因此跳2

                intakeLinks = "https://www.fatsecret.cn/" + html_intakeLinks[i]["href"] # 取得"食物名"連結
                response = requests.get(intakeLinks)
                html = BeautifulSoup(response.text) 

                brand = html.find_all("div", class_ = "breadcrumb_link") # 該食物有品牌，為[2]；若無，為""
                size =  html.find_all("div", class_ = "serving_size_value") # 食用量[0]
                intake =  html.find_all("div", class_ = "breadcrumb_noLink") # 食物名

                if len(brand) == 3: # 該食物"有"品牌

                    intakeLinks_set.add((brand[1].text, iintake[0].text, size[0].text , intakeLinks))
                    print("品牌："+ brand[2].text + "，食物名：" + intake[0].text + 
                          "，食用量：" + size[0].text + "，網址：", intakeLinks)  

                else: # 該食物"無"品牌

                    intakeLinks_set.add(("NULL", intake[0].text, size[0].text , intakeLinks))
                    print("品牌："+ "NULL" + "，食物名：" + intake[0].text + 
                          "，食用量：" + size[0].text + "，網址：", intakeLinks)  

    except:
        errorLinks_set.add((iLink))

# 有錯誤連結則印，無也印        
if len(errorLinks_set) != 0: 
    for i in errorLinks_set:
        print(("錯誤連結：", list(i))
else:
    print("沒有錯誤連結")

# 儲存檔案
# 如果沒有該檔案名，創資料夾
dn = "fatsecret_food"
if not os.path.exists(dn):   
    os.makedirs(dn)    

# 將食用量連結連結存檔
print("食用量連結存檔中...")

intakeLinks_df = pd.DataFrame(intakeLinks_set)
intakeLinks_df.to_csv(dn + "/" + 'WebLinks_fatsecret_3_intakeLinks_csv', index=False, header=False, encoding="utf-8")

print("食用量連結存檔（csv）完成")

    
# 將錯誤連結存檔
print("錯誤連結存檔中...")

errorLinks_df = pd.DataFrame(errorLinks_set)
errorLinks_df.to_csv(dn + "/" + 'WebLinks_fatsecret_3_errorLinks_scv', index=False, header=False, encoding="utf-8")

print("錯誤連結存檔（csv）完成")


# 第四部分：爬蟲

In [ ]:
# set()要轉換成list()方能套入引數
intakeLinks_set = list(intakeLinks_set)

print("創建空字典")
# 設個空字串將所有食物字典塞入
final_dic = []

# 將食物名稱，營養成分設個空字典
saved = {} # 食物名稱字典
nutrient_dic = {} # 營養成分字典



for link in range(len(intakeLinks_set)):
    
    print("抓取食物名，食用量，營養成分等...") #檢查哨 
    print("")
    url = intakeLinks_set[link][3] #在set的第三項
    response = requests.get(url)  # 輸入cookie及內容
    html = BeautifulSoup(response.text)

    ## 抓品牌，食物名，食用量，營養成分等
    # 品牌 
    brand = html.find_all("div", class_ = "breadcrumb_link") # 該食物有品牌，為[2]；若無，為""
    if len(brand) == 3: # 該食物"有"品牌
        braned = brand[2].text
        print("品牌："+ brand[2].text)
              
    else: # 該食物"無"品牌
        braned = "NULL"
        print("品牌："+ "NULL" )
    
    
    # 食物名 
    name = html.find_all("div", class_="breadcrumb_noLink")
    print("食物名: "+name[0].text) # 檢查哨  

    # 食用量
    intake = html.find_all("div", class_="serving_size")

    for i in range(len(intake)): # 跑回圈只是為了格式好看 # 檢查哨
        print(intake[i].text, end = " ")

    print(" ") # 為格式好看

    # 營養成分 # 印出營養成分 # 檢查哨
    nutrient = html.find_all("div", class_="nutrient")
    for n in range(1, 3, 2):
        print("-" + nutrient[n].text, nutrient[n+1].text) # -為格式好看

    # ## 300 千卡的前一項(3)為空值，補上“卡路里”
    # 補上卡路里、印出卡路里
    print("-" + "卡路里 ",nutrient[4].text) # -為格式好看
    
    # 印出其他營養成分
    for n in range(5, len(nutrient), 2):
        print("-" + nutrient[n].text, nutrient[n+1].text) # -為格式好看
    
    # 印出更新時間，並將時間轉換成8碼，如20190911
    time = html.find("div", style ="margin-top:5px").find("div", class_ = "smallText").text
    timeDrop = time[8:19].replace('年','').replace('月','').replace('日','')
    print("更新時間：", timeDrop)
    print("資訊來源:fatsecret中国")
    
    print("")

    # 先整合成一個物件
    # 食物名先創字典 (saved)

    print("創字典...") #檢查哨
    print("")

    saved = {"品牌":braned ,
             "食物名":name[0].text,
             intake[0].text:intake[1].text,
            }
    
    # 創空字典
    nutrient_dic= {} 
    
    # update營養成分到字典

    for i in range (1, 3, 2):
        dic = {nutrient[i].text: nutrient[i+1].text} 
        nutrient_dic.update(dic)
        
    # update卡路里到字典
    dic2 = {"熱量":nutrient[4].text}
    nutrient_dic.update(dic2)
    
    # update其他營養成分到字典
    for n in range(5, len(nutrient), 2):
        dic3 = {nutrient[n].text:nutrient[n+1].text}
        nutrient_dic.update(dic3)

    # update"資料來源：fatsecret中國"到字典
    
    dic_source= {"資訊來源":"fatsecret中国"}
    nutrient_dic.update(dic_source)
    
    # update"更新時間：timeDrop"到字典
    
    dic_time= {"更新時間": timeDrop}
    nutrient_dic.update(dic_time)
        
        
    # update到最初的字典
    saved.update(nutrient_dic)
    
    print("該食物加入字典完成")
    print("")
    final_dic.append((saved))
    
print("="*90)    
print("字典創建完成")

# 第五部分：存檔

In [ ]:
## 存檔

print("存檔中...") #檢查哨
print("")

dn = "fatsecret_food"
fn = "fatsecret_foodData"
if not os.path.exists(dn):   
    os.makedirs(dn)
if not os.path.exists(dn+"/"+fn):   
    os.makedirs(dn+"/"+fn)
    
# 存json格式
f = open(dn + "/"+ fn +"/"+ "Data_fatsecret" + ".json", "w", encoding="utf-8") # 打開路徑
json.dump(final_dic, f) # 寫入json格式
f.close() # 關閉路徑
print("json存檔完成")
print("")

# 存csv格式
final_dic_df = pd.DataFrame(final_dic)
final_dic_df.to_csv(dn + "/"+ fn +"/"+ "Data_fatsecret" + ".csv", index=False, header=False, encoding="utf-8")

print("csv存檔完成")
print("")

# # DONE
print("存檔完成")
print("="*90)

print("Done!")
            

# 備用
#### 若不幸於第一~三部分中斷，可直接讀特定部分的csv檔，以利續爬

import csv

### 開啟 CSV 檔案
with open('fatsecret_food\\3_fatsecret_intakeLinks_csv', newline='', encoding="utf-8") as csvfile:

  ### 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    intakeLinks_set = [] # 設空字串，以便後續爬
    
    # 以迴圈輸出每一列
    for row in rows:
        print(row)
        intakeLinks_set.append(row) # 將csv每行逐一加入字串

for i in range (len(intakeLinks_set)):
    print(i, intakeLinks_set[i])